In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install langchain ultralytics 'unstructured[pdf]' opencv-python-headless==4.8.1.78 python-dotenv tiktoken openai==0.28.1 chromadb==0.3.29

# fist add Libgl1
!sudo apt-get update
!sudo apt-get install libgl1 -y

# second add sql
!pip install --user pysqlite3-binary
# then in settings.py:
# these three lines swap the stdlib sqlite3 lib with the pysqlite3 
!package__import__('pysqlite3')
import sys
!sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
# SUPRESS warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI

In [ ]:
# @title Default title text
# All Azure deployment information is stored in a .env file
import os
from io import StringIO
from dotenv import load_dotenv
load_dotenv(r'/home/rvanderwil/data/volume_2/config.env')
#import cv2

In [ ]:
loader = UnstructuredFileLoader(r'/home/rvanderwil/data/volume_2/Sample.pdf')
documents = loader.load()

#display(documents)

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=8000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

#display(texts)

In [ ]:
#### https://python.langchain.com/docs/integrations/text_embedding/azureopenai

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="EMBEDDING",
    openai_api_version = "2023-09-15-preview",
)

doc_search = Chroma.from_documents(texts,embeddings);
chain = RetrievalQA.from_chain_type(llm=AzureOpenAI(model_kwargs={'engine':'DAVINCI'}),chain_type='stuff', retriever = doc_search.as_retriever());


In [ ]:
query = 'summarize'
chain.run(query)

In [ ]:
query = 'who are tested and how?'
chain.run(query)